## Units, Scaling, non-dimensionalization

* We assume that the reader is familiar with the rationale of non-dimensionalisation. 
* explain potential temp convention

In [2]:
import numpy as np
from easydict import EasyDict as edict

#this does't actually need to be protected. More a reminder it's an interim measure
import sys
try:
    sys.path.append('../../../unsupported')
except:
    pass

import warnings; warnings.simplefilter('ignore')
from unsupported_dan.UWsubduction.minimal_example import paramDict_dim, modelDict_dim, UnitRegistry

## Pint / Scaling

In the UWsubduction module, we make use of the Scaling module, which allows us to work with a dimensional quantities, amd automates the non-dimensionlisation process. 

The scaling module (Romain Beucher) is a simple wrapper around the Pint library:

https://github.com/hgrecco/pint

https://github.com/underworldcode/underworld2/blob/master/docs/examples/1_14_ScalingExample.ipynb

https://github.com/rbeucher/UWGeodynamics/tree/master/UWGeodynamics/scaling

Throughout the UWsubduction module, we will store paramters in dictionaries. More specifically, we use `edicts` which allow attribute-like access to items. In fact, we've already imported a set of parameters in an edict:

In [3]:
print(type(paramDict_dim))
print(paramDict_dim.refDensity, paramDict_dim['refDensity'])

<class 'easydict.EasyDict'>
(<Quantity(3300.0, 'kilogram / meter ** 3')>, <Quantity(3300.0, 'kilogram / meter ** 3')>)


Notice that the dictionary key 'refDensity' has a corresponding value which is a Pint 'Quantity', i.e it has both magnitude and units. 

We can of course define new dimensional quantities:

In [4]:
someVal = 42.*UnitRegistry.British_horsepower

We can define a system of scaling factors, and use this to non-dimensionlize our paramter set. Let's set one of these systems up:

In [5]:
import unsupported_dan.geodynamics.scaling as test_scaling;

In [6]:
print(test_scaling.scaling)

TransformedDict({'[temperature]': <Quantity(1.0, 'kelvin')>, '[time]': <Quantity(1.0, 'second')>, '[length]': <Quantity(1.0, 'meter')>, '[mass]': <Quantity(1.0, 'kilogram')>, '[substance]': <Quantity(1.0, 'mole')>})


In [9]:
print(paramDict_dim.refLength)
print(test_scaling.nonDimensionalize(paramDict_dim.refLength))

2900 kilometer
2900000.0


Because the default length scale is 1 meter, all that happenened is that the Pint Quantity was transoformed inoto it;s base unit and divided by the scaling factor, leavinf the dimensionless value 2900000.0

## Scaling of mantle thermo-mechanical subduction

Now we will import a system of scaling factors that are commonly used related to thermal convection/ subduction studies. This system uses the convective layer depth (the mantle) as the length scale, a thermal diffusion times cale, and a stress (& mass) scale based on a viscous stress.

length Scale:
$L_s = L_r$

time scale: $t_s = \frac{L_s^2}{\kappa_r}$

stress scale $\sigma_s = \frac{\kappa_r \eta_r}{L_s}$

mass scale: $M_s = \frac{\eta_r L_s^3}{\kappa_r}$

Note that we only need the stress __or__ the mass scale to define a complete scaling


In [10]:
from unsupported_dan.UWsubduction.default_scaling import sub_scaling

In [12]:
print(sub_scaling.nonDimensionalize(paramDict_dim.refLength))

1.0


We also provide a function to convert a dictionary of dimensional pramters to a dictionary of dimesnsionless parameters:

In [20]:
from unsupported_dan.UWsubduction.default_scaling import build_nondim_dict

paramDict_nondim = build_nondim_dict(paramDict_dim, sub_scaling)

In [21]:
print(paramDict_nondim.refLength)

1.0


## Temperature convention (offset)

One difficulty with allowing the scaling module to automate the non-dimensionalisation process is that we sometimes want to define an offset as well as a scaling. For intaance, it is common in thermal (Raylieigh-Bernard type) convection studies to deine the dimensionless temperature as:

$T' = \frac{T - T_0}{T_1 -T_0}$ 

Where $T_0$ is the surface temperature and T1, the base (or interior) temperature. 

At this stage, we have tired to adhere to this convention, by providing two sets of temperatures:


In [16]:
print(paramDict_dim.potentialTemp)
print(paramDict_dim.potentialTemp_)

1573.0 kelvin
1300.0 kelvin


In [17]:
print(sub_scaling.nonDimensionalize(paramDict_dim.potentialTemp))
print(sub_scaling.nonDimensionalize(paramDict_dim.potentialTemp_))

1.21
1.0


Where the `_` notation desribes the offset temperatures (dimensional and dimensionless)

We mention this to clarify (hopefully) the appearance of `potentialTemp_` and `surfaceTemp_` in the examples.

## Scaling Arrhennius Laws